In [4]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir
    path.append(dir(path[0])+'/dash_app')
    __package__ = "dash_app"

In [5]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map, map_columns, stations_map_hovertemplate
from settings import *


DATA_PATH: /Users/dan/ClimateIreland/CI-Status-Report/CI-Status-Report-Dash/data/


In [6]:
data_path = DATA_PATH+'Terrestrial_Domain/4.9AboveGroundBiomass/Figure4.17/'
data_csv = data_path + 'Figure4.17_data.csv'

In [8]:
"""
Tidy data for charts
"""
columns_dict = {
               'ownership':'Unnamed: 0',
               '2006_m3':"1000's m3",
                '2012_m3':"1000's m3.1",
                '2017_m3':"1000's m3.2",
                '2006_%':"%",
                '2012_%':"%.1",
                '2017_%':"%.2",
                  }
xls = pd.ExcelFile(
    data_path+'TotalGrowinStockVolume_byCounty.xlsx')
original_df = pd.read_excel(xls, 'GrowingStockVolumeByOwnership', skiprows=[0,1,7])
df = map_columns(columns_dict, original_df)
df_06 = pd.DataFrame()
df_06['ownership']=df['ownership']
df_06['sum__annual__growing_stock_volume']=df['2006_m3']
df_06['percentage__annual__growing_stock_volume']=df['2006_%']
df_06['datetime']='2006'
df_12 = pd.DataFrame()
df_12['ownership']=df['ownership']
df_12['sum__annual__growing_stock_volume']=df['2012_m3']
df_12['percentage__annual__growing_stock_volume']=df['2012_%']
df_12['datetime']='2012'
df_17 = pd.DataFrame()
df_17['ownership']=df['ownership']
df_17['sum__annual__growing_stock_volume']=df['2017_m3']
df_17['percentage__annual__growing_stock_volume']=df['2017_%']
df_17['datetime']='2017'
df = pd.concat([df_06,df_12,df_17])
df['datetime'] = pd.to_datetime(df['datetime'],format='%Y')
df['sum__annual__growing_stock_volume']=df['sum__annual__growing_stock_volume']
df['location'] = 'Ireland'
df=df[[
    'datetime',
    'ownership',
    'sum__annual__growing_stock_volume',
    'percentage__annual__growing_stock_volume',
    'location']]
df.reset_index()
# df.to_csv(data_csv) #commented out as the file should be kept as is

,Unnamed: 0,1000's m3,%,1000's m3.1,%.1,1000's m3.2,%.2
0,Public,51.713,0.719625,60.405,0.619691,64.783,0.555963
1,Private (grant aided),7.490,0.104229,21.824,0.223891,34.090,0.292558
2,Private (Other),12.658,0.176146,15.247,0.156418,17.651,0.151480
3,Total,71.861,1.000000,97.476,1.000000,116.524,1.000000


In [12]:

df

,datetime,ownership,sum__annual__growing_stock_volume,percentage__annual__growing_stock_volume,location
0,2006-01-01,Public,51.713,0.719625,Ireland
1,2006-01-01,Private (grant aided),7.490,0.104229,Ireland
2,2006-01-01,Private (Other),12.658,0.176146,Ireland
3,2006-01-01,Total,71.861,1.000000,Ireland
0,2012-01-01,Public,60.405,0.619691,Ireland
1,2012-01-01,Private (grant aided),21.824,0.223891,Ireland
2,2012-01-01,Private (Other),15.247,0.156418,Ireland
3,2012-01-01,Total,97.476,1.000000,Ireland
0,2017-01-01,Public,64.783,0.555963,Ireland
1,2017-01-01,Private (grant aided),34.090,0.292558,Ireland


In [25]:
data_path = DATA_PATH+'Terrestrial_Domain/4.9AboveGroundBiomass/Figure4.17/'
data_csv = data_path + 'Figure4.17_data.csv'
df = pd.read_csv(data_csv, index_col=0)
df['datetime'] = pd.to_datetime(df['datetime'])
public_df = df.loc[(df['ownership'] == 'Public')]
private_grants_df = df.loc[(df['ownership'] == 'Private (grant aided)')]
private_other_df = df.loc[(df['ownership'] == 'Private (Other)')]
total_df = df.loc[(df['ownership'] == 'Total')]
total_trace = go.Scatter(x=total_df['datetime'].dt.year,
                         y=total_df['sum__annual__growing_stock_volume'],
                         name='Total',
                         text=total_df['sum__annual__growing_stock_volume'],
                                 mode='lines',  # 'line' is default
                         line_shape='spline',
                         line=dict(
                                # color="#fc0d1b", color used in report
                                color=TIMESERIES_COLOR_1,
                                width=2),
                         hovertemplate='<b>Total</b><br>' +
                         'Stock Volume: %{y:.2f} Mm<sup>3</sup><br>' +
                         '------------------------------<br>' +
                         '<extra></extra>'
                         )
public_trace = go.Scatter(x=public_df['datetime'].dt.year,
                         y=public_df['sum__annual__growing_stock_volume'],
                         name='Public',
                         text=public_df['percentage__annual__growing_stock_volume'],
                         marker=dict(color='#5b9bd5'), # blue,
                         stackgroup="one",
                         hovertemplate='<b>Public</b><br>' +
                         'Stock Volume: %{y:.2f} Mm<sup>3</sup><br>' +
                         'Percent of Total: %{text:.2%}<br>' +
                         '<extra></extra>'
                         )
private_grant_trace = go.Scatter(x=private_grants_df['datetime'].dt.year,
                         y=private_grants_df['sum__annual__growing_stock_volume'],
                         name='Private (Grant Aided)',
                         text=private_grants_df['percentage__annual__growing_stock_volume'],
                         marker=dict(color='#ed7d31'), # brown,
                         stackgroup="one",
                         hovertemplate='<b>Private (Grant Aided)</b><br>' +
                         'Stock Volume: %{y:.2f} Mm<sup>3</sup><br>' +
                         'Percent of Total: %{text:.2%}<br>' +
                         '<extra></extra>'
                         )
private_other_trace = go.Scatter(x=private_other_df['datetime'].dt.year,
                         y=private_other_df['sum__annual__growing_stock_volume'],
                         name='Private (Other)',
                         text=private_other_df['percentage__annual__growing_stock_volume'],
                         marker=dict(color='#ffff00'), # yellow,
                         stackgroup="one",
                         hovertemplate='<b>Private (Other)</b><br>' +
                         'Stock Volume: %{y:.2f} Mm<sup>3</sup><br>' +
                         'Percent of Total: %{text:.2%}<br>' +
                         '<extra></extra>'
                         )
figure_4_17 = go.Figure(data=[public_trace,private_other_trace,private_grant_trace,total_trace],
                        layout=TIMESERIES_LAYOUT)
figure_4_17.update_layout(
        yaxis=dict(title='Total Growing Stock Volume (million m<sup>3</sup>)'),
    xaxis=dict(title="Year",tickvals=[2006, 2012, 2017],range=[2005, 2018]),
    hovermode="x unified",)
figure_4_17